In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import wandb

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# initial value

GWP_0 = 8.5e13 #initial world GDP, used by takeoff model
RH_0=1.7e11 #initial research input to hardware (only relevant for exogeneous modelling)
RS_0=1.7e10 #initial research input to software (only releveant for exogeneous modelling)
HS_0 =  2e28 #https://docs.google.com/document/d/1rw1pTbLi2brrEP0DcsZMAVhlKp6TKGKNUSFRkkdP_hs/edit#bookmark=id.b15qyylbeqxp
K_0= 6.1e14 #cumulative sum of depreciation adjusted capital investments
L_0 = 4e9
QH_0=2.2e9 #cumulative hardware research (USD) at start of simulation, read from graph (unsure how this is determined)
QS_0=1e8 #at start of simulation, no software research done
SK_0=0.5 #initial capital factor share
SCog_0=0.5 #initial cognitive factor share
SC_0=0.01 #initial factor share of compute in cognitive output production
SL_0=0.99  #initial factor share of labour in cognitive output production


S_0=1 
H_0=1.5e17 #needs to be updated
C_eff_0=HS_0*S_0 #initial stocks of effective compute



In [ ]:
#### utils

def construct_automation_thresholds(threshold_100,training_FLOP_GAP):
    ##NOTE: the actual FTM sets 0,5,10,20 and 50 by hand then interpolates in log space between these

    threshold_20=threshold_100-training_FLOP_GAP
    threshold_0=threshold_20-(0.5*training_FLOP_GAP)
    requirements_0_20=np.logspace(threshold_0,threshold_20,21)
    requirements_20_100=np.logspace(threshold_20,threshold_100,80)
    training_requirements=np.concatenate((requirements_0_20[:-1],requirements_20_100),axis=0)

    return training_requirements

def construct_baseline_runtime_efficiencies(runtime_reqs_100,runtime_FLOP_GAP):

    runtime_reqs_20=runtime_reqs_100-runtime_FLOP_GAP
    runtime_reqs_0=runtime_reqs_20-(0.5*runtime_FLOP_GAP)
    runtime_reqs_0_20 = np.logspace(runtime_reqs_0,runtime_reqs_20,21)
    runtime_reqs_20_100 = np.logspace(runtime_reqs_20,runtime_reqs_100,80)
    runtime_reqs = np.concatenate((runtime_reqs_0_20[:-1],runtime_reqs_20_100),axis=0)

    return runtime_reqs

In [3]:
# parameter dump


HS_depreciation_rate = 0.2 #hardware stock deprecitation rate
s_K = 0.2 #savings rate - 0.2/year
f_GWP_H = 8e-4 #fraction of world output spent on hardware
TFP = 1 #Total factor productivity - grows exogeneously in FTM (not implemented yet)
T = 30 #number of years to run simulation for
delta_T=1 #size of time increment. n_steps=T/delta_T
n_steps = T/delta_T
g_L = 0.01


#### hardware research parameters 
alpha_H = 0.0092 #factor share of capital in hardware research production (used when shares are NOT computed)
KS_H=None #capital share in hardware research productions
CogS_H=None #cognitive share in hardware production
f_K_H = 2e-3 # 2e-3 #fraction of capital stock for hardware research production
f_L_H = 2e-3 #2e-3 #fraction of labour stock for hardware research production
f_C_H = 2e-3
f_GWP_H=2e-3 #t=0 fraction of GWP spent on hardware research (~ $140B on semiconductor R&D in 2022)
rho_H = 1 #substitution parameter between capital and cogntiive labour for hardware research production
rH = 7.4286 #hardware efficiency returns to hardware research - determined from comparing historical data on hardware (H) growth and research input ($) growth
lambda_H = 0.7 #'stepping on toes' effect for hardware research (i.e: How much research per unit time contributes to hardware stock)
ratio_initial_cumulative_hardware_input = 0.063 #ratio of 2022 hardware input to cumulative hardware input (in $)


#### software research parameters 
alpha_S = 0.2571 #factor share of capital in software research production
KS_S=None #capital share in software research productions
CogS_S=None #cognitive share in software research production
f_K_S = 2e-4 #2e-4 fraction of capital stock for software research production
f_L_S = 2e-4 #2e-4 #fraciton of labour stock for software research production
f_C_S = 2e-4
f_GWP_S = 2e-4 #only needed for t=0. Payments to software researchers estimated as ~$10-20b, which is approx. 0.02% of GWP
rho_S = 1 #substitution parameter between capital and cognitive labour for software research production
rS = 1.7857
ratio_initial_cumulative_software_input = 0.2 #ratio of initial software inputs to cumulative software research inputs

#### GWP production
alpha_G = 0.0002 #task weight non-compute capital in GWP production - should be computed each iteration
KS_G=None #capital share in goods productions
CogS_G=None #cognitive share in goods production
f_K_GWP = 1-f_K_H-f_K_S #fraction of capital stock for producing GWP - not modelling hardware and software research endogeneously right now
f_L_GWP = 1-f_L_H-f_L_S #fraction of labour for producing GWP - not modelling hardware and software research endogeneously right now
f_C_GWP = 1-f_C_H-f_C_S #fraction of effective compute for producing GWP - not modelling hardware and software research endogeneously right now
rho_G = -0.4 #substitution parameter between capital and cognitive labour for GWP production
psi_G = -0.5 #substitution parameter between cognitive tasks in cognitive input production



#### automation
n_labour_tasks=100 #fixed for both production and research

#automation - training
f_C_T = 1.6e-4 #fraction of compute assigned to frontier training run in a year
automation_training_task100 = 36 #compute threshold for last task (logspace)
training_FLOP_GAP=4
automation_training_threshold=construct_automation_thresholds(automation_training_task100,training_FLOP_GAP)

#automation - runtime
eta_0 = 6.0e23
persecond_runtime_100 = 16 + 1 - np.log10(6) #bioanchors is 1e16 FLOP/s, adjust up by 1 OOM to account for TAI v.s. AGI, and down by 6x for AGI one-time benefits compared to human
runtime_100 = persecond_runtime_100 + np.log10(60*60*24*365)
runtime_FLOP_GAP=1
baseline_runtime_requirements = construct_baseline_runtime_efficiencies(runtime_100,runtime_FLOP_GAP)
delta=1.5 # Scaling factor of runtime efficiency C_T less than/greater than runtime requirements



#### checks
assert f_K_GWP+f_K_H+f_K_S==1
assert f_L_GWP+f_L_H+f_L_S==1
assert f_C_GWP+f_C_H+f_C_S==1

/Users/iyngkarrankumar/opt/anaconda3/envs/scalingtrends/lib/python3.11/site-packages/numpy/core/function_base.py:298: RuntimeWarning: overflow encountered in power
  return _nx.power(base, y)


## Main loop

In [4]:

#logging setup
LOGGING=False
logging_dict = {
    't':None,
    'year':None,

    'GWP (2022 $)':None,
    'HS (FLOP/year)':None,
    'C_eff (2022 FLOP/year)':None,
    'H (FLOP/$/year)':None,
    'S (2022 FLOP/FLOP/year)':None,
    'RS (2022 $)':None,

    'Cog_G (2022 $)':None, #still no idea how to physically interpret this variable
    'Automation index':None,
    'C_eff_T':None,

    'K (2022 $)':None,
    'L (HLY/year)':None, #HLH - human labour years
    'Capital to cognitive share ratio':None,
    'alpha_G':None,
    'Compute to labour share ratio':None,
    'beta_0':None,
    'beta_i':None,

    'RH (2022 $)':None,
    'QH (2022 $)':None,
    'g_H':None,
    'g_QH':None,

    'RS (2022 $)':None,
    'QS (2022 $)':None,
    'g_S':None,
    'g_QS':None,
}



#### bools

##production bools
GWP_ENDOGENEOUS,exg_g_GWP=True,0.02
H_ENDOGENEOUS,g_H=False,0.5 #use derivate method of Jones 1995 research modeling, if not hardware (H) exogeneous
S_ENDOGENEOUS,g_S=False,0.5 #use derivate method of Jones 1995 research modeling, if not software (S) exogeneous
RH_ENDOGENEOUS,ex_g_RH=False,0.02 #produce hardware research each time step, if not hardware production (RH) exogeneous
RS_ENDOGENEOUS,ex_g_RS=False,0.02 #produce software research each time step, if not  software production (RS) exogeneous
COGNITIVES_INPUT_ENDOGENOUS,g_Cog=True,0.02 #'Produce' (calculate) cognitive inputs each time step, if not cognitive inputs (Cog) exogeneous
COMPUTE_OUTER_WEIGHTS,alpha_G=True,0.0002 #compute outer weights to CES production function, if not constant
COMPUTE_INNER_WEIGHTS,beta_0,beta_i = True,1,4e-11 #compute inner weights to CES production function, if not constant
COMPUTE_RUNTIME_EFFICIENCIES = True


##Research bools
HARDWARE_PENALTY_FACTOR=False #apply penalty factor to hardware efficiency growth rate - leave for now
SOFTWARE_PENALTY_FACTOR=False #apply penalty factor to software efficiency growth rate - leave for now

automation_index=0

### main loop
if LOGGING:
    run=wandb.init(project="FTM-implementation")

#the BL is a wonderful training environment

for t in range (int(n_steps)):
    year=2022+t

    if t==0: #### initial

        ###set initial state variables
        GWP = GWP_0 
        HS =  HS_0 
        C_eff=C_eff_0 #initial stocks of effective compute
        S=S_0
        H=H_0

        Cog_G=None
        automation_index=None
        C_T=f_C_T*C_eff

        K = K_0 
        L = L_0

        RH_0=f_GWP_H*GWP_0
        RH=RH_0
        QH_0=(RH_0**lambda_H)/ratio_initial_cumulative_hardware_input ##FTM also multiplies by 1/lambda_H but idk why
        QH=QH_0
        g_QH=None 

        RS_0=f_GWP_S*GWP_0
        RS=RS_0
        QS_0=(RS**lambda_H)/ratio_initial_cumulative_software_input ##FTM also multiplies by 1/lambda_H but idk why
        QS=QS_0
        g_QS=None

    else: #### update factors of production

        #### non output variables
        g_K=(s_K*GWP)/K
        K=(1+g_K)*K #depreciation not implemented yet

        #L 
        L = (1+g_L)*L
    

    if 1: #### ALLOCATIONS (static for now in this model)

        #allocating compute
        C_eff_GWP=f_C_GWP*C_eff #effective compute in GWP production

        #allocating labour
        L_GWP=f_L_GWP*L
        L_H=f_L_H*L
        L_HS=f_L_S*L

        #allocating capital
        K_GWP=f_K_GWP*K
        K_H=f_K_H*K
        K_S=f_K_S*K

        #allocating compute 
        C_eff_GWP=f_C_GWP*C_eff
        C_eff_H=f_C_H*C_eff
        C_eff_S=f_C_S*C_eff
        C_T= f_C_T*C_eff
        
    
    if 1: #### set automation conditions (not differentiating between GWP and research for now)

        C_T = f_C_T * C_eff #largest training run in this timestep (assuming training runs are instant)

        ####automation (only done for G&S production)
        automation_bools=automation_training_threshold<=C_T 
        automated_task_indices=np.arange(1,n_labour_tasks+1)[automation_bools]
        if len(automated_task_indices)==0: automation_index=0
        else:automation_index=automated_task_indices[-1]




    if GWP_ENDOGENEOUS: #production and GWP

        if COGNITIVES_INPUT_ENDOGENOUS:
            labour_per_task=L_GWP/n_labour_tasks #not allocating optimally yet

            effective_compute_per_task = C_eff_GWP/(automation_index+1) #not allocating optimially, and not distinguishing between train and runtime effective compute

            #labour and compute arrays
            labour_array = np.concatenate(\
                (np.zeros(1),np.ones(n_labour_tasks) * labour_per_task),\
                axis=0
                )
            effective_compute_array = np.concatenate( \
                (np.ones(automation_index+1)*effective_compute_per_task,np.zeros(n_labour_tasks-automation_index)),\
                axis=0
                )

            if COMPUTE_RUNTIME_EFFICIENCIES:
                eta_0 = 1/baseline_runtime_requirements
                training_requirements_multiplier = (C_T/automation_training_threshold)**delta
                eta = training_requirements_multiplier*eta_0

                _eta_ = np.concatenate(\
                    (np.ones(1),eta),
                    axis=0) #_eta_ adds the eta=1 for the compute only task


            else:
                    eta = np.ones(n_labour_tasks)*eta_0
                    _eta_ = np.concatenate(\
                        (np.ones(1),eta),
                        axis=0) #_eta_ adds the eta=1 for the compute only task



            #inner weights
            if COMPUTE_INNER_WEIGHTS:
                if t==0:
                    compute_to_labour_ratio=SC_0/SL_0
                else:
                    compute_to_labour_ratio = (beta_0/(n_labour_tasks*beta_i))*((C_eff_GWP*n_labour_tasks)/L_GWP)**(psi_G)
                    omega_= compute_to_labour_ratio * ((L_GWP/(n_labour_tasks*C_eff_GWP))**psi_G)
                    beta_0 = omega_/(1+omega_)
                    beta_i = (1-beta_0)/n_labour_tasks

                inner_weights = np.concatenate(\
                    (np.ones(1)*beta_0,np.ones(n_labour_tasks)*beta_i),\
                    axis=0
                    )

            else:
                inner_weights = np.concatenate(\
                    (np.ones(1)*beta_0,np.ones(n_labour_tasks)*beta_i),\
                    axis=0
                    )
            
            task_outputs = labour_array + (_eta_*effective_compute_array)
            Cog_G = (np.sum(inner_weights*(task_outputs)**(psi_G)))**(1/psi_G) #compute->labour efficiencies NOT YET IMPLEMENTED
        else:
            Cog_G = 2.0e24

        if COMPUTE_OUTER_WEIGHTS:
            if t==0:
                capital_to_cognitive_ratio=SK_0/SCog_0
            else:
                capital_to_cognitive_ratio = (alpha_G/(1-alpha_G))*((K_GWP/Cog_G)**rho_G)
            omega=capital_to_cognitive_ratio*((Cog_G/K_GWP)**rho_G)
            alpha_G=omega/(1+omega)
        else:
            alpha_G=alpha_G


        production_gwp = TFP * (alpha_G*K_GWP**rho_G + (1-alpha_G)*Cog_G**rho_G)**(1/rho_G)

        if t==0: #all we need is gwp_to_production ratio
            gwp_to_production_ratio = GWP/production_gwp
        else:
            GWP=gwp_to_production_ratio * production_gwp

    else:
        if t==0: pass
        else: GWP=(1+exg_g_GWP)*GWP #change GWP exogeneously


    if H_ENDOGENEOUS:

        
                #hardware/software research production (only relevant if )
        if RH_ENDOGENEOUS:
            
            if COGNITIVES_INPUT_ENDOGENOUS:

                labour_per_task = L_H/n_labour_tasks #divide labour force for hardware research uniformly between tasks (not doing optimal allocation right now)
                effective_compute_per_task = C_eff_H/n_labour_tasks
                
                #labour and compute arrays
                labour_array = np.concatenate(\
                    (np.zeros(1),np.ones(n_labour_tasks) * labour_per_task),\
                    axis=0
                    )
                effective_compute_array = np.concatenate( \
                    (np.ones(automation_index+1)*effective_compute_per_task,np.zeros(n_labour_tasks-automation_index)),\
                    axis=0
                    )

            else:
                Cog_H = 4.0e21 # but what does 'cognitive input' mean?

            if COMPUTE_OUTER_WEIGHTS:
                if t==0:
                    pass
                else:
                    pass
            else:
                alpha_H=alpha_H

            production_hardware = TFP * (alpha_H*K_H**rho_H + (1-alpha_H)*Cog_H**rho_H)**(1/rho_H)

            if t==0:
                RH_to_production_ratio  = RH/production_hardware
            else:
                RH = RH * production_hardware

            if COMPUTE_OUTER_WEIGHTS: #Not implemented yet
                pass
            else:
                alpha_H=alpha_H
            
            production_hardware = TFP * (alpha_H*K_H**rho_H + (1-alpha_H)*Cog_H**rho_H)**(1/rho_H)

            if t==1:
                hardware_to_production_ratio = RH_0/production_hardware

            RH = hardware_to_production_ratio * production_hardware
                
        else: RH=(1+ex_g_RH)*RH

        g_QH=(delta_T*(RH**lambda_H))/QH
        QH=(1+g_QH)*QH

        if HARDWARE_PENALTY_FACTOR:
            pass
        else:
            g_H=rH*g_QH

    else:

        if t==0: pass
        else:
            g_QH=None #no cumulative hardware research required
            H=(1+g_H)*H


    if S_ENDOGENEOUS:

        if RS_ENDOGENEOUS:

            if COMPUTE_OUTER_WEIGHTS:
                pass #not implemented
            else:
                alpha_S=alpha_S
            
            if COGNITIVES_INPUT_ENDOGENOUS: #not implemented
                pass
            else:
                Cog_S = 4.0e20 #

            production_software = TFP * (alpha_S*K_S**rho_S + (1-alpha_S)*Cog_S**rho_S)**(1/rho_S)

            if t==1:
                software_to_production_ratio = RS_0/production_software

            RS = software_to_production_ratio * production_software

        else: RS=(1+ex_g_RS)*RS

        g_QS=(delta_T*(RS**lambda_H))/QS #stepping on toes parameter same for hardware and software research (man the abstractness of growth models is beautiful!)
        QS=(1+g_QS)*QS

        if SOFTWARE_PENALTY_FACTOR:
            pass
        else:
            g_S=rS*g_QS

    else:
        if t==0: pass
        else:
            g_QS=None
            S=(1+g_S)*S

    #hardware stock and effective compute update 
    if t==0:
        pass #do nothing on first pass
    else:

        g_HS = (f_GWP_H*GWP*H - HS_depreciation_rate*HS)/HS
        HS = (1+g_HS)*HS; 

        C_eff = HS*S; 

    #logging
    if LOGGING:
        #we do this at the start to log initial values of variable
        logging_dict['t']=t
        logging_dict['year']=year

        logging_dict['GWP (2022 $)']=GWP
        logging_dict['HS (FLOP/year)']=HS
        logging_dict['C_eff (2022 FLOP/year)']=C_eff
        logging_dict['H (FLOP/$/year)']=H
        logging_dict['S (2022 FLOP/FLOP/year)']=S

        logging_dict['Cog_G (2022 $)']=Cog_G
        logging_dict['Automation index']=automation_index
        logging_dict['C_eff_T']=C_T

        logging_dict['K (2022 $)']=K
        logging_dict['L (HLY/year)']=L
        logging_dict['Capital to cognitive share ratio']=capital_to_cognitive_ratio
        logging_dict['alpha_G']=alpha_G
        logging_dict['Compute to labour share ratio']=compute_to_labour_ratio
        logging_dict['beta_0']=beta_0
        logging_dict['beta_i']=beta_i

        logging_dict['RH (2022 $)']=RH
        logging_dict['QH (2022 USD)']=QH
        logging_dict['g_H']=g_H
        logging_dict['g_QH']=g_QH

        logging_dict['RS (2022 $)']=RS
        logging_dict['QS (2022 USD)']=QS
        logging_dict['g_S']=g_S
        logging_dict['g_QS']=g_QS


        run.log(logging_dict)
        
if LOGGING:
    wandb.finish()


print('Finish')

Finish


## Old stuff and experiments

In [ ]:
## Hardware research production - NOT IN USE
delta_t = 1
r_H = 7.4286 #hardware efficiency returns to hardware R&D
R_Hs = []


#computing growth in cumulative research output
K_H = f_K_H * K 
Cog_H = 4e21 #constant for now

R_H = TFP * (alpha_H*K_H**rho_H + (1-alpha_H)*Cog_H**rho_H)**(1/rho_H)
R_Hs = [R_H]*10 #for now, this is how we have list of annual hardware research


Q_H_new = sum(R_Hs)
Q_H_dot = (difference in Q_H)/delta_t

g_QH = Q_H_dot/Q_H

#computing penalty factor to efficiency growth rate
'''
intuition: Growth rate in technological (hardware or software)
efficency should be modulated by a penalty factor. 
'''





In [ ]:
## Software research production - NOT IN USE

K_S = f_K_S * K 
Cog_S = f_Cog_S * Cog

R_S = TFP * (alpha_S*K_S**rho_S + (1-alpha_S)*Cog_S**rho_S)**(1/rho_S)

In [ ]:
## Production function experiments

TFP,alpha_G,K_GWP,Cog_GWP,rho_G = 1,0.0002,6.1e14,2.0e28,-0.4
Y_1=alpha_G*K_GWP+(1-alpha_G)*Cog_GWP
Y_2=alpha_G*K_GWP**rho_G+(1-alpha_G)*Cog_GWP**rho_G
Y_3=(alpha_G*K_GWP**rho_G+(1-alpha_G)*Cog_GWP**rho_G)**(1/rho_G)
Y_4=(K_GWP**rho_G + Cog_GWP**rho_G)**(1/rho_G)
print(Y_1/1e13,Y_2,Y_3/1e13,Y_4/1e13)

In [ ]:
## checking factor shares
KS,CogS = 0.5,0.5 #sum must equal to 1
K_G,Cog_G,GWP,rho_G=6.1e14,2e24,8.5e13,-0.4
omega_G=(KS/CogS) * ((Cog_G/K_G)**rho_G)
alpha_G=omega_G/1+omega_G ###pretty much checks out
print(alpha_G)


## checking for capital share
'''
We know: KS = alpha * (K/Y)**rho_G
'''
alpha = KS*(GWP/K_G)**rho_G ##doesn't match alpha above
print(alpha)

Y_ = (alpha*(K_G**rho_G) + (1-alpha)*(Cog_G**rho_G))**(1/rho_G)

## checking for cognitive share
'''
We know: CogS = (1-alpha)*(Cog_G/Y)**rho_G
'''
beta = CogS*((GWP/Cog_G)**rho_G)
alpha=1-beta
print(alpha)


##working backward
'''
Plugging in values for GWP, K_G, Cog_G, rho_G, apparently alpha = 2.199892. 
Idk I'm really confused here.
'''



In [ ]:
## Cognitive output production - NOT IN USE

n_labour_tasks=9
automation_index = 5
S=1
HS=2e24
C_eff=HS*S #effective compute (not factoring in a train-runtime traedeoff)
L=4e9  #global workforce
psi=-0.5 #substitution parameter amongst tasks for cognitive output production function
eta_0 = 6e-23 #initial compute to labour efficiency

#calculating task outputs - assuming uniform compute and labour allocation so far
labour_allocation=[0]+list(np.repeat(L/n_labour_tasks,n_labour_tasks))
task_automation_index = [1]+list(np.ones(automation_index,dtype=int)) + list(np.zeros(n_labour_tasks-automation_index,dtype=int))

compute_allocation=list(np.repeat(C/(automation_index+1),automation_index+1)) + list(np.zeros(n_labour_tasks-automation_index))
compute_to_labour_efficiencies=[1]+list(np.repeat(eta_0,n_labour_tasks)) #efficiences - constant for now
zipped=list(zip(labour_allocation,compute_to_labour_efficiencies,compute_allocation))

task_outputs=[tup[0]+tup[1]*tup[2] for tup in zipped]


#computing cognitive outputs
task_weights=list(np.repeat(1/(n_labour_tasks+1),n_labour_tasks+1)) #all tasks equally important for now
weight_output_products=list((tup[0]*(tup[1]**psi) for tup in list(zip(task_weights,task_outputs))))
Cog = sum(weight_output_products)**(1/psi)




In [5]:
n_rows=int(np.ceil(np.sqrt(len(plot_dict))))
fig,axs_matrix=plt.subplots(figsize=(10,10),nrows=n_rows,ncols=n_rows)

plot_num=0
for ax_row in axs_matrix:
    for ax in ax_row:
        plot_key=list(plot_dict.keys())[plot_num]
        ax.plot(plot_dict['t'],plot_dict[plot_key])
        ax.set_ylabel(plot_key)

        if plot_key=='t': ax.set_yscale('linear')
        else: ax.set_yscale('log')
        
        plot_num+=1

        if plot_num>=len(plot_dict):
            break
    if plot_num>=len(plot_dict):
            break


fig.tight_layout()


NameError: name 'plot_dict' is not defined

In [ ]:



    if t==0: 

        ###allocations

        #allocating compute
        C_eff_GWP=f_C_GWP*C_eff #effective compute in GWP production

        #allocating labour
        L_GWP=f_L_GWP*L
        L_H=f_L_H*L
        L_HS=f_L_S*L

        #allocating capital
        K_GWP=f_K_GWP*K
        K_H=f_K_H*K
        K_S=f_K_S*K


        ####set automation conditions
        C_T = f_C_T * C_eff #largest training run in this timestep (assuming training runs are instant)

        ####automation (only done for G&S production)
        automation_bools=automation_training_threshold<=C_T 
        automated_task_indices=np.arange(1,n_labour_tasks+1)[automation_bools]
        if len(automated_task_indices)==0: automation_index=0
        else:automation_index=automated_task_indices[-1]

        #done to compute gwp-to-production factor (need to do for all produced quantities)
        if GWP_ENDOGENEOUS: #production and GWP

            if COGNITIVES_INPUT_ENDOGENOUS:
                labour_per_task=L_GWP/n_labour_tasks #not allocating optimally yet

                effective_compute_per_task = C_eff_GWP/(automation_index+1) #not allocating optimially, and not distinguishing between train and runtime effective compute

                #labour and compute arrays
                labour_array = np.concatenate(\
                    (np.zeros(1),np.ones(n_labour_tasks) * labour_per_task),\
                    axis=0
                    )
                effective_compute_array = np.concatenate( \
                    (np.ones(automation_index+1)*effective_compute_per_task,np.zeros(n_labour_tasks-automation_index)),\
                    axis=0
                    )

                if COMPUTE_RUNTIME_EFFICIENCIES:
                    eta_0 = 1/base_runtime_requirements
                    training_requirements_multiplier = (C_T/automation_training_threshold)**delta
                    eta = training_requirements_multiplier*eta_0

                    _eta_ = np.concatenate(\
                        (np.ones(1),eta),
                        axis=0) #_eta_ adds the eta=1 for the compute only task


                else:
                        eta = np.ones(n_labour_tasks)*eta_0
                        _eta_ = np.concatenate(\
                            (np.ones(1),eta),
                            axis=0) #_eta_ adds the eta=1 for the compute only task



                #inner weights
                if COMPUTE_INNER_WEIGHTS:
                    compute_to_labour_ratio=SC_0/SL_0
                    omega_= compute_to_labour_ratio * ((L_GWP/(n_labour_tasks*C_eff_GWP))**psi_G)
                    beta_0 = omega_/(1+omega_)
                    beta_i = (1-beta_0)/n_labour_tasks

                    inner_weights = np.concatenate(\
                        (np.ones(1)*beta_0,np.ones(n_labour_tasks)*beta_i),\
                        axis=0
                        )

                else:
                    inner_weights = np.concatenate(\
                        (np.ones(1)*beta_0,np.ones(n_labour_tasks)*beta_i),\
                        axis=0
                        )
                
                task_outputs = labour_array + (_eta_*effective_compute_array)
                Cog_G = (np.sum(inner_weights*(task_outputs)**(psi_G)))**(1/psi_G) #compute->labour efficiencies NOT YET IMPLEMENTED
            else:
                Cog_G = 2.0e24

            if COMPUTE_OUTER_WEIGHTS:
                capital_to_cognitive_ratio=SK_0/SCog_0
                omega=capital_to_cognitive_ratio*((Cog_G/K_GWP)**rho_G)
                alpha_G=omega/(1+omega)
            else:
                alpha_G=alpha_G


            production_gwp = TFP * (alpha_G*K_GWP**rho_G + (1-alpha_G)*Cog_G**rho_G)**(1/rho_G)

            if t==0:
                gwp_to_production_ratio = GWP/production_gwp
            else:
                GWP=gwp_to_production_ratio * production_gwp

            GWP=GWP

        else: GWP=GWP




    ####### IF T NOT EQUAL 0
    else: 



            
        #allocating compute
        C_eff_GWP=f_C_GWP*C_eff #effective compute in GWP production

        #allocating labour
        L_GWP=f_L_GWP*L
        L_H=f_L_H*L
        L_HS=f_L_S*L

        #allocating capital
        K_GWP=f_K_GWP*K
        K_H=f_K_H*K
        K_S=f_K_S*K

        ####set automation conditions
        C_T = f_C_T * C_eff #largest training run in this timestep (assuming training runs are instant)



        ####automation (only done for G&S production)
        automation_bools=automation_training_threshold<=C_T 
        automated_task_indices=np.arange(1,n_labour_tasks+1)[automation_bools]
        if len(automated_task_indices)==0: automation_index=0
        else:automation_index=automated_task_indices[-1]

        #### production
        if GWP_ENDOGENEOUS: #production and GW

            if COGNITIVES_INPUT_ENDOGENOUS:
                labour_per_task=L_GWP/n_labour_tasks #not allocating optimally yet

                ###compute HLH-to-compute effiencies
                effective_compute_per_task = C_eff_GWP/(automation_index+1) #not allocating optimially, and not distinguishing between train and runtime effective compute

                #labour and compute arrays
                labour_array = np.concatenate(\
                    (np.zeros(1),np.ones(n_labour_tasks) * labour_per_task),\
                    axis=0
                    )
                effective_compute_array = np.concatenate( \
                    (np.ones(automation_index+1)*effective_compute_per_task,np.zeros(n_labour_tasks-automation_index)),\
                    axis=0,
                    )

                if COMPUTE_RUNTIME_EFFICIENCIES:
                    eta_0 = 1/base_runtime_requirements
                    training_requirements_multiplier = (C_T/automation_training_threshold)**delta
                    eta = training_requirements_multiplier*eta_0

                    _eta_ = np.concatenate(\
                        (np.ones(1),eta),
                        axis=0) #_eta_ adds the eta=1 for the compute only task

                else:
                        eta = np.ones(n_labour_tasks)*eta_0
                        _eta_ = np.concatenate(\
                            (np.ones(1),eta),
                            axis=0)



                #inner weights
                if COMPUTE_INNER_WEIGHTS:
                    compute_to_labour_ratio = (beta_0/n_labour_tasks*beta_i)*(C_eff_GWP*n_labour_tasks/L_GWP)**(psi_G)
                    omega = compute_to_labour_ratio * ((L_GWP/n_labour_tasks*C_eff_GWP)**psi_G)
                    beta_0 = omega_/(1+omega_)
                    beta_i = (1-beta_0)/n_labour_tasks

                    inner_weights = np.concatenate(\
                        (np.ones(1)*beta_0,np.ones(n_labour_tasks)*beta_i),\
                        axis=0
                        )

                else:
                    inner_weights = np.concatenate(\
                        (np.ones(1)*beta_0,np.ones(n_labour_tasks)*beta_i),\
                        axis=0
                        )

                task_outputs = labour_array + (_eta_*effective_compute_array)
                Cog_G = (np.sum(inner_weights*(task_outputs)**(psi_G)))**(1/psi_G)

            else:
                Cog_G = (1+g_Cog)*Cog_G

            if COMPUTE_OUTER_WEIGHTS:

                ##compute new task share ratio
                capital_to_cognitive_ratio = (alpha_G/(1-alpha_G))*((K_GWP/Cog_G)**rho_G)
                omega=capital_to_cognitive_ratio*((Cog_G/K_GWP)**rho_G)
                alpha_G=omega/(1+omega)

            else:
                alpha_G=alpha_G

            production_gwp = TFP * (alpha_G*K_GWP**rho_G + (1-alpha_G)*Cog_G**rho_G)**(1/rho_G)

            GWP=gwp_to_production_ratio * production_gwp #normalise production
            
        else: GWP=(1+exg_g_GWP)*GWP


        if H_ENDOGENEOUS:

                    #hardware/software research production (only relevant if )
            if RH_ENDOGENEOUS:

                if COMPUTE_OUTER_WEIGHTS: #Not implemented yet
                    pass
                else:
                    alpha_H=alpha_H
                
                if COGNITIVES_INPUT_ENDOGENOUS: #not implemented
                    Cog_H=None 
                else:
                    Cog_H = 4.0e21 # but what does 'cognitive input' mean?

                production_hardware = TFP * (alpha_H*K_H**rho_H + (1-alpha_H)*Cog_H**rho_H)**(1/rho_H)

                if t==1:
                    hardware_to_production_ratio = RH_0/production_hardware

                RH = hardware_to_production_ratio * production_hardware
                    
            else: RH=(1+ex_g_RH)*RH

            g_QH=(delta_T*(RH**lambda_H))/QH
            QH=(1+g_QH)*QH

            if HARDWARE_PENALTY_FACTOR:
                pass
            else:
                g_H=rH*g_QH

        else:
            g_QH=None #no cumulative hardware research required
            H=(1+g_H)*H

        if S_ENDOGENEOUS:

            if RS_ENDOGENEOUS:

                if COMPUTE_OUTER_WEIGHTS:
                    pass #not implemented
                else:
                    alpha_S=alpha_S
                
                if COGNITIVES_INPUT_ENDOGENOUS: #not implemented
                    pass
                else:
                    Cog_S = 4.0e20 #

                production_software = TFP * (alpha_S*K_S**rho_S + (1-alpha_S)*Cog_S**rho_S)**(1/rho_S)

                if t==1:
                    software_to_production_ratio = RS_0/production_software

                RS = software_to_production_ratio * production_software

            else: RS=(1+ex_g_RS)*RS

            g_QS=(delta_T*(RS**lambda_H))/QS #stepping on toes parameter same for hardware and software research (man the abstractness of growth models is beautiful!)
            QS=(1+g_QS)*QS

            if SOFTWARE_PENALTY_FACTOR:
                pass
            else:
                g_S=rS*g_QS

        else:
            g_QS=None
            S=(1+g_S)*S

        #hardware stock and effective compute update 
        
        g_HS = (f_GWP_compute*GWP*H - HS_depreciation_rate*HS)/HS
        HS = (1+g_HS)*HS; 

        C_eff = HS*S; 